In [4]:
"""
1.数据：
    a.图片
    b.随机向量
2.构建计算图：
    a.生成器
    b.判别器
    c.DCGAN
        连接g和d
        定义loss
        定义train_op
3.训练流程

"""
import os
import sys
import tensorflow as tf
from tensorflow import logging
from tensorflow import gfile
import pprint
import numpy as np
import random
import math
from PIL import Image 
from tensorflow.examples.tutorials.mnist import input_data#直接导入mnist数据集

mnist = input_data.read_data_sets('MNIST_data/', one_hot = True)

output_dir = './local_run'
if not gfile.Exists(output_dir):
    gfile.MakeDirs(output_dir)

def get_default_params():
    return tf.contrib.training.HParams(
        z_dim = 100,#随机向量长度
        init_conv_size = 4,#将随机向量反卷积，要先把向量转成矩阵（特征图），初始大小为4
        g_channels = [128, 64, 32, 1],#给生成器的反卷积通道数，4层反卷积，通道数递减，在后面用统一的卷积核和步长
        d_channels = [32, 64, 128, 256],#给判别器的卷积层通道数，步长为2，不做pooling
        batch_size = 128,
        learning_rate = 0.002,
        beta1 = 0.5,#
        img_size = 32,#生成图像的长宽32*32
    )

hps = get_default_params()

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [5]:
#数据生成器的实现
class MnistData(object):
    def __init__(self, mnist_train, z_dim, img_size):#训练数据集，向量长度，生成图片的长度
        self._data = mnist_train
        self._example_num = len(self._data)
        self._z_data = np.random.standard_normal((self._example_num, z_dim))#给每个样本生成一个随即向量
        self._indicator = 0#用于batch
        self._resize_mnist_img(img_size)#28*28 -> 32*32
        self._random_shuffle()
    
    def _random_shuffle(self):
        p = np.random.permutation(self._example_num)
        self._z_data = self._z_data[p]
        self._data = self._data[p]
    
    def _resize_mnist_img(self, img_size):
        """
        1.先把numpy转成PIL的image
        2.PIL的image去resize
        3.再转回numpy
        """
        data = np.asarray(self._data * 255, np.uint8)#mnist做过归一化
        data = data.reshape((self._example_num, 1, 28, 28))
        data = data.transpose((0, 2, 3, 1))
        new_data = []
        for i in range(self._example_num):
            img = data[i].reshape((28, 28))
            img = Image.fromarray(img)#把numpy矩阵变成pil对象
            img = img.resize((img_size, img_size))#resize
            img = np.asarray(img)#变回numpy
            img = img.reshape((img_size, img_size, 1))
            new_data.append(img)
        new_data = np.asarray(new_data, dtype=np.float32)#列表变成大矩阵
        new_data = new_data / 127.5 - 1#归一到-1至1
        self._data = new_data
        
    def next_batch(self, batch_size):
        end_indicator = self._indicator + batch_size
        if end_indicator > self._example_num:
            self._random_shuffle()
            self._indicator = 0
            end_indicator = self._indicator + batch_size
        assert end_indicator < self._example_num
        
        batch_data = self._data[self._indicator: end_indicator]
        batch_z = self._z_data[self._indicator: end_indicator]
        self._indicator = end_indicator
        return batch_data, batch_z

mnist_data = MnistData(mnist.train.images, hps.z_dim, hps.img_size)
batch_data, batch_z = mnist_data.next_batch(5)
print(batch_data.shape)
print(batch_z.shape)

(5, 32, 32, 1)
(5, 100)


In [7]:
def conv2d_transpose(inputs, out_channel, name, training, with_bn_relu=True):
#实现反卷积的封装（输入，输出的通道，名字，是否可训练，最后一层不需要relu）
    with tf.variable_scope(name):
        conv2d_trans = tf.layers.conv2d_transpose(inputs, out_channel,[5, 5],strides=(2,2),padding='SAME')
        if with_bn_relu:
            bn = tf.layers.batch_normalization(conv2d_trans, training=training)
            relu = tf.nn.relu(bn)
            return relu
        else:
            return conv2d_trans

def conv2d(inputs, out_channel, name, training):#卷积层的封装
    def leaky_relu(x, leak = 0.2, name = ''):
        return tf.maximum(x, x * leak, name = name)
    with tf.variable_scope(name):
        conv2d_output = tf.layers.conv2d(inputs,out_channel,[5, 5],strides = (2, 2), padding = 'SAME')
        bn = tf.layers.batch_normalization(conv2d_output,training = training)
        return leaky_relu(bn, name = 'outputs')


class Generator(object):
    def __init__(self, channels, init_conv_size):
        assert len(channels) > 1
        self._channels = channels
        self._init_conv_size = init_conv_size
        self._reuse = False#生成器判别器可能不止被使用一次
    
    def __call__(self, inputs, training):#把类的对象的构建当成一个函数用
        inputs = tf.convert_to_tensor(inputs)#把numpy数组变成tensor
        with tf.variable_scope('generator', reuse=self._reuse):
            with tf.variable_scope('inputs'):
                fc = tf.layers.dense(inputs,self._channels[0] * self._init_conv_size * self._init_conv_size)
                #初始向量 -> fc -> channel[0]*init_conv_size**2 -> reshape (init_couv_size,init_conv_size,channel[0])
                conv0 = tf.reshape(fc, [-1, self._init_conv_size, self._init_conv_size, self._channels[0]])
                bn0 = tf.layers.batch_normalization(conv0, training=training)
                relu0 = tf.nn.relu(bn0)
            
            deconv_inputs = relu0
            # deconvolutions * 4
            for i in range(1, len(self._channels)):
                with_bn_relu = (i != len(self._channels) - 1)#判断是否是最后一层
                deconv_inputs = conv2d_transpose(deconv_inputs,#反卷积
                                                 self._channels[i],
                                                 'deconv-%d' % i,
                                                 training,
                                                 with_bn_relu)
            img_inputs = deconv_inputs
            with tf.variable_scope('generate_imgs'):
                # imgs value scope: [-1, 1]
                imgs = tf.tanh(img_inputs, name='imgaes')#最后一层的激活函数
        self._reuse=True
        self.variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='generator')
        #生成器判别器分别优化，获得某个scope下的所有的参数便于保存
        return imgs
    
        
class Discriminator(object):
    def __init__(self, channels):
        self._channels = channels
        self._reuse = False
    
    def __call__(self, inputs, training):
        inputs = tf.convert_to_tensor(inputs, dtype=tf.float32)
        
        conv_inputs = inputs
        with tf.variable_scope('discriminator', reuse = self._reuse):
            for i in range(len(self._channels)):
                conv_inputs = conv2d(conv_inputs,
                                     self._channels[i],
                                     'conv-%d' % i,
                                     training)
            fc_inputs = conv_inputs
            with tf.variable_scope('fc'):
                flatten = tf.layers.flatten(fc_inputs)
                logits = tf.layers.dense(flatten, 2, name="logits")#输出类别个数2
        self._reuse = True
        self.variables = tf.get_collection(tf.GraphKeys.TRAINABLE_VARIABLES, scope='discriminator')
        return logits


class DCGAN(object):
    def __init__(self, hps):
        g_channels = hps.g_channels
        d_channels = hps.d_channels
        self._batch_size = hps.batch_size
        self._init_conv_size = hps.init_conv_size
        self._batch_size = hps.batch_size
        self._z_dim = hps.z_dim
        self._img_size = hps.img_size
        
        self._generator = Generator(g_channels, self._init_conv_size)
        self._discriminator = Discriminator(d_channels)
    
    def build(self):
        self._z_placholder = tf.placeholder(tf.float32, (self._batch_size, self._z_dim))
        self._img_placeholder = tf.placeholder(tf.float32,(self._batch_size, self._img_size, self._img_size, 1))#真实图像的placeholder
        
        generated_imgs = self._generator(self._z_placholder, training = True)#生成图像的placeholder
        fake_img_logits = self._discriminator(generated_imgs, training = True)
        real_img_logits = self._discriminator(self._img_placeholder, training = True)
        
        loss_on_fake_to_real = tf.reduce_mean(#生成器损失函数：让判别器把假的图片判断成真的
            tf.nn.sparse_softmax_cross_entropy_with_logits(
                labels = tf.ones([self._batch_size], dtype = tf.int64),
                logits = fake_img_logits))
        loss_on_fake_to_fake = tf.reduce_mean(#判别器损失函数：把假的判断成假的损失函数
            tf.nn.sparse_softmax_cross_entropy_with_logits(
                labels = tf.zeros([self._batch_size], dtype = tf.int64),
                logits = fake_img_logits))
        loss_on_real_to_real = tf.reduce_mean(#判别器损失函数：把真的判断成真的损失函数
            tf.nn.sparse_softmax_cross_entropy_with_logits(
                labels = tf.ones([self._batch_size], dtype = tf.int64),
                logits = real_img_logits))
        
        tf.add_to_collection('g_losses', loss_on_fake_to_real)#给一个key，先存入，再可以通过这个key取出
        tf.add_to_collection('d_losses', loss_on_fake_to_fake)
        tf.add_to_collection('d_losses', loss_on_real_to_real)
        
        
        loss = {
            'g': tf.add_n(tf.get_collection('g_losses'), name = 'total_g_loss'),#g的损失函数是把所有g_losses的损失函数都加起来
            'd': tf.add_n(tf.get_collection('d_losses'), name = 'total_d_loss')#g的损失函数是把所有d_losses的损失函数都加起来
        }
        
        return (self._z_placholder, self._img_placeholder, generated_imgs, loss)
    
    def build_train(self, losses, learning_rate, beta1):
        g_opt = tf.train.AdamOptimizer(learning_rate = learning_rate, beta1 = beta1)
        d_opt = tf.train.AdamOptimizer(learning_rate = learning_rate, beta1 = beta1)
        g_opt_op = g_opt.minimize(losses['g'], var_list = self._generator.variables)
        d_opt_op = d_opt.minimize(losses['d'], var_list = self._discriminator.variables)
        with tf.control_dependencies([g_opt_op, d_opt_op]):#交叉训练
            return tf.no_op(name = 'train')


dcgan = DCGAN(hps)
z_placeholder, img_placeholder, generated_imgs, losses = dcgan.build()
train_op = dcgan.build_train(losses, hps.learning_rate, hps.beta1)

In [ ]:
def combine_and_show_imgs(batch_imgs, img_size, rows=8, cols=16):#把很多图片展示在一张大图上，batch=8*16
    # batch_imgs: [batch_size, img_size, img_size, 1]
    result_big_img = []
    for i in range(rows):
        row_imgs = []
        for j in range(cols):
            img = batch_imgs[cols * i + j]#把小图展示成大图的过程
            img = img.reshape((img_size, img_size))#img_size, img_size, 1 -> img_size, img_size
            img = (img + 1) * 127.5
            row_imgs.append(img)
        row_imgs = np.hstack(row_imgs)
        result_big_img.append(row_imgs)
    result_big_img = np.vstack(result_big_img)
    result_big_img = np.asarray(result_big_img, np.uint8)
    result_big_img = Image.fromarray(result_big_img)#把矩阵变成图像
    return result_big_img

    
    
init_op = tf.global_variables_initializer()
train_steps = 10000

with tf.Session() as sess:
    sess.run(init_op)
    for step in range(train_steps):
        batch_img, batch_z = mnist_data.next_batch(hps.batch_size)
        
        fetches = [train_op, losses['g'], losses['d']]
        should_sample = (step + 1) % 50 == 0
        if should_sample:
            fetches += [generated_imgs]#生成一次图像
        out_values = sess.run(fetches,
                              feed_dict = {
                                  z_placeholder: batch_z,
                                  img_placeholder: batch_img
                              })
        _, g_loss_val, d_loss_val = out_values[0:3]
        logging.info('step: %d, g_loss: %4.3f, d_loss: %4.3f' % (step, g_loss_val, d_loss_val))
        if should_sample:
            gen_imgs_val = out_values[3]
            
            gen_img_path = os.path.join(output_dir, '%05d-gen.jpg' % (step + 1))
            gt_img_path = os.path.join(output_dir, '%05d-gt.jpg' % (step + 1))
            
            gen_img = combine_and_show_imgs(gen_imgs_val, hps.img_size)
            gt_img = combine_and_show_imgs(batch_img, hps.img_size)
            
            gen_img.save(gen_img_path)
            gt_img.save(gt_img_path)

INFO:tensorflow:step: 0, g_loss: 0.727, d_loss: 1.852
INFO:tensorflow:step: 1, g_loss: 0.004, d_loss: 9.052
INFO:tensorflow:step: 2, g_loss: 28.878, d_loss: 14.333
INFO:tensorflow:step: 3, g_loss: 24.426, d_loss: 0.124
INFO:tensorflow:step: 4, g_loss: 5.680, d_loss: 0.062
INFO:tensorflow:step: 5, g_loss: 0.000, d_loss: 12.537
INFO:tensorflow:step: 6, g_loss: 16.294, d_loss: 0.021
INFO:tensorflow:step: 7, g_loss: 14.032, d_loss: 0.180
INFO:tensorflow:step: 8, g_loss: 5.335, d_loss: 0.128
INFO:tensorflow:step: 9, g_loss: 0.016, d_loss: 4.529
INFO:tensorflow:step: 10, g_loss: 10.055, d_loss: 3.641
INFO:tensorflow:step: 11, g_loss: 8.230, d_loss: 1.095
INFO:tensorflow:step: 12, g_loss: 4.295, d_loss: 0.033
INFO:tensorflow:step: 13, g_loss: 1.042, d_loss: 0.491
INFO:tensorflow:step: 14, g_loss: 1.790, d_loss: 0.220
INFO:tensorflow:step: 15, g_loss: 1.537, d_loss: 0.299
INFO:tensorflow:step: 16, g_loss: 5.029, d_loss: 0.073
INFO:tensorflow:step: 17, g_loss: 1.280, d_loss: 0.675
INFO:tensorfl

INFO:tensorflow:step: 149, g_loss: 1.957, d_loss: 0.366
INFO:tensorflow:step: 150, g_loss: 1.788, d_loss: 0.555
INFO:tensorflow:step: 151, g_loss: 0.320, d_loss: 1.830
INFO:tensorflow:step: 152, g_loss: 8.709, d_loss: 4.365
INFO:tensorflow:step: 153, g_loss: 5.681, d_loss: 0.423
INFO:tensorflow:step: 154, g_loss: 2.044, d_loss: 0.299
INFO:tensorflow:step: 155, g_loss: 0.973, d_loss: 0.692
INFO:tensorflow:step: 156, g_loss: 4.893, d_loss: 0.096
INFO:tensorflow:step: 157, g_loss: 3.412, d_loss: 0.323
INFO:tensorflow:step: 158, g_loss: 0.162, d_loss: 2.494
INFO:tensorflow:step: 159, g_loss: 6.768, d_loss: 4.026
INFO:tensorflow:step: 160, g_loss: 1.513, d_loss: 0.675
INFO:tensorflow:step: 161, g_loss: 0.483, d_loss: 1.148
INFO:tensorflow:step: 162, g_loss: 2.769, d_loss: 0.459
INFO:tensorflow:step: 163, g_loss: 3.104, d_loss: 0.322
INFO:tensorflow:step: 164, g_loss: 2.032, d_loss: 0.296
INFO:tensorflow:step: 165, g_loss: 1.256, d_loss: 0.470
INFO:tensorflow:step: 166, g_loss: 2.301, d_loss

INFO:tensorflow:step: 296, g_loss: 2.348, d_loss: 0.218
INFO:tensorflow:step: 297, g_loss: 2.282, d_loss: 0.281
INFO:tensorflow:step: 298, g_loss: 2.006, d_loss: 0.325
INFO:tensorflow:step: 299, g_loss: 1.952, d_loss: 0.435
INFO:tensorflow:step: 300, g_loss: 2.244, d_loss: 0.610
INFO:tensorflow:step: 301, g_loss: 0.274, d_loss: 1.842
INFO:tensorflow:step: 302, g_loss: 8.492, d_loss: 2.942
INFO:tensorflow:step: 303, g_loss: 1.716, d_loss: 0.462
INFO:tensorflow:step: 304, g_loss: 1.182, d_loss: 0.611
INFO:tensorflow:step: 305, g_loss: 3.904, d_loss: 0.221
INFO:tensorflow:step: 306, g_loss: 4.189, d_loss: 0.260
INFO:tensorflow:step: 307, g_loss: 2.242, d_loss: 0.257
INFO:tensorflow:step: 308, g_loss: 1.612, d_loss: 0.362
INFO:tensorflow:step: 309, g_loss: 2.718, d_loss: 0.280
INFO:tensorflow:step: 310, g_loss: 2.318, d_loss: 0.353
INFO:tensorflow:step: 311, g_loss: 1.774, d_loss: 0.438
INFO:tensorflow:step: 312, g_loss: 2.632, d_loss: 0.297
INFO:tensorflow:step: 313, g_loss: 2.285, d_loss

INFO:tensorflow:step: 443, g_loss: 1.801, d_loss: 0.372
INFO:tensorflow:step: 444, g_loss: 2.135, d_loss: 0.615
INFO:tensorflow:step: 445, g_loss: 0.918, d_loss: 0.753
INFO:tensorflow:step: 446, g_loss: 3.818, d_loss: 1.087
INFO:tensorflow:step: 447, g_loss: 0.597, d_loss: 1.300
INFO:tensorflow:step: 448, g_loss: 4.944, d_loss: 0.515
INFO:tensorflow:step: 449, g_loss: 3.787, d_loss: 0.162
INFO:tensorflow:step: 450, g_loss: 0.978, d_loss: 0.715
INFO:tensorflow:step: 451, g_loss: 4.003, d_loss: 0.576
INFO:tensorflow:step: 452, g_loss: 1.759, d_loss: 0.360
INFO:tensorflow:step: 453, g_loss: 1.846, d_loss: 0.359
INFO:tensorflow:step: 454, g_loss: 2.965, d_loss: 0.195
INFO:tensorflow:step: 455, g_loss: 1.873, d_loss: 0.595
INFO:tensorflow:step: 456, g_loss: 1.096, d_loss: 0.915
INFO:tensorflow:step: 457, g_loss: 3.717, d_loss: 1.028
INFO:tensorflow:step: 458, g_loss: 0.186, d_loss: 2.371
INFO:tensorflow:step: 459, g_loss: 7.766, d_loss: 2.522
INFO:tensorflow:step: 460, g_loss: 3.509, d_loss

INFO:tensorflow:step: 590, g_loss: 1.657, d_loss: 0.547
INFO:tensorflow:step: 591, g_loss: 1.720, d_loss: 0.424
INFO:tensorflow:step: 592, g_loss: 2.788, d_loss: 0.474
INFO:tensorflow:step: 593, g_loss: 0.467, d_loss: 1.419
INFO:tensorflow:step: 594, g_loss: 6.806, d_loss: 3.278
INFO:tensorflow:step: 595, g_loss: 0.811, d_loss: 0.987
INFO:tensorflow:step: 596, g_loss: 1.762, d_loss: 0.439
INFO:tensorflow:step: 597, g_loss: 3.157, d_loss: 0.496
INFO:tensorflow:step: 598, g_loss: 2.212, d_loss: 0.412
INFO:tensorflow:step: 599, g_loss: 1.018, d_loss: 0.802
INFO:tensorflow:step: 600, g_loss: 2.956, d_loss: 0.632
INFO:tensorflow:step: 601, g_loss: 1.289, d_loss: 0.628
INFO:tensorflow:step: 602, g_loss: 2.469, d_loss: 0.874
INFO:tensorflow:step: 603, g_loss: 0.930, d_loss: 0.822
INFO:tensorflow:step: 604, g_loss: 3.572, d_loss: 0.792
INFO:tensorflow:step: 605, g_loss: 1.062, d_loss: 0.724
INFO:tensorflow:step: 606, g_loss: 2.609, d_loss: 0.472
INFO:tensorflow:step: 607, g_loss: 2.189, d_loss

...

INFO:tensorflow:step: 2902, g_loss: 1.989, d_loss: 0.415
INFO:tensorflow:step: 2903, g_loss: 1.783, d_loss: 0.647
INFO:tensorflow:step: 2904, g_loss: 2.229, d_loss: 0.528
INFO:tensorflow:step: 2905, g_loss: 1.154, d_loss: 0.698
INFO:tensorflow:step: 2906, g_loss: 1.739, d_loss: 0.473
INFO:tensorflow:step: 2907, g_loss: 2.746, d_loss: 0.730
INFO:tensorflow:step: 2908, g_loss: 1.144, d_loss: 0.680
INFO:tensorflow:step: 2909, g_loss: 3.085, d_loss: 0.464
INFO:tensorflow:step: 2910, g_loss: 2.084, d_loss: 0.459
INFO:tensorflow:step: 2911, g_loss: 1.276, d_loss: 0.492
INFO:tensorflow:step: 2912, g_loss: 3.015, d_loss: 0.422
INFO:tensorflow:step: 2913, g_loss: 2.778, d_loss: 0.362
INFO:tensorflow:step: 2914, g_loss: 1.258, d_loss: 0.549
INFO:tensorflow:step: 2915, g_loss: 2.553, d_loss: 0.882
INFO:tensorflow:step: 2916, g_loss: 0.893, d_loss: 0.805
INFO:tensorflow:step: 2917, g_loss: 3.849, d_loss: 0.591
INFO:tensorflow:step: 2918, g_loss: 2.155, d_loss: 0.461
INFO:tensorflow:step: 2919, g_l

INFO:tensorflow:step: 3046, g_loss: 1.763, d_loss: 0.483
INFO:tensorflow:step: 3047, g_loss: 2.061, d_loss: 0.419
INFO:tensorflow:step: 3048, g_loss: 1.695, d_loss: 0.724
INFO:tensorflow:step: 3049, g_loss: 1.998, d_loss: 0.317
INFO:tensorflow:step: 3050, g_loss: 1.617, d_loss: 0.573
INFO:tensorflow:step: 3051, g_loss: 1.945, d_loss: 0.665
INFO:tensorflow:step: 3052, g_loss: 0.993, d_loss: 0.926
INFO:tensorflow:step: 3053, g_loss: 3.641, d_loss: 1.449
INFO:tensorflow:step: 3054, g_loss: 0.402, d_loss: 1.498
INFO:tensorflow:step: 3055, g_loss: 4.097, d_loss: 0.682
INFO:tensorflow:step: 3056, g_loss: 2.184, d_loss: 0.405
INFO:tensorflow:step: 3057, g_loss: 1.485, d_loss: 0.519
INFO:tensorflow:step: 3058, g_loss: 1.962, d_loss: 0.511
INFO:tensorflow:step: 3059, g_loss: 1.485, d_loss: 0.604
INFO:tensorflow:step: 3060, g_loss: 2.769, d_loss: 0.332
INFO:tensorflow:step: 3061, g_loss: 2.024, d_loss: 0.543
INFO:tensorflow:step: 3062, g_loss: 1.441, d_loss: 0.513
INFO:tensorflow:step: 3063, g_l

INFO:tensorflow:step: 3190, g_loss: 2.842, d_loss: 0.561
INFO:tensorflow:step: 3191, g_loss: 1.229, d_loss: 0.592
INFO:tensorflow:step: 3192, g_loss: 2.666, d_loss: 0.311
INFO:tensorflow:step: 3193, g_loss: 3.021, d_loss: 0.262
INFO:tensorflow:step: 3194, g_loss: 1.902, d_loss: 0.397
INFO:tensorflow:step: 3195, g_loss: 2.662, d_loss: 0.405
INFO:tensorflow:step: 3196, g_loss: 1.776, d_loss: 0.320
INFO:tensorflow:step: 3197, g_loss: 2.850, d_loss: 0.434
INFO:tensorflow:step: 3198, g_loss: 1.523, d_loss: 0.492
INFO:tensorflow:step: 3199, g_loss: 2.730, d_loss: 0.344
INFO:tensorflow:step: 3200, g_loss: 2.890, d_loss: 0.222
INFO:tensorflow:step: 3201, g_loss: 2.078, d_loss: 0.236
INFO:tensorflow:step: 3202, g_loss: 3.742, d_loss: 0.628
INFO:tensorflow:step: 3203, g_loss: 0.687, d_loss: 0.896
INFO:tensorflow:step: 3204, g_loss: 4.521, d_loss: 0.478
INFO:tensorflow:step: 3205, g_loss: 2.486, d_loss: 0.334
INFO:tensorflow:step: 3206, g_loss: 2.660, d_loss: 0.200
INFO:tensorflow:step: 3207, g_l

INFO:tensorflow:step: 3334, g_loss: 1.808, d_loss: 0.430
INFO:tensorflow:step: 3335, g_loss: 1.930, d_loss: 0.352
INFO:tensorflow:step: 3336, g_loss: 3.580, d_loss: 0.191
INFO:tensorflow:step: 3337, g_loss: 2.612, d_loss: 0.431
INFO:tensorflow:step: 3338, g_loss: 1.184, d_loss: 0.628
INFO:tensorflow:step: 3339, g_loss: 4.193, d_loss: 0.740
INFO:tensorflow:step: 3340, g_loss: 1.701, d_loss: 0.389
INFO:tensorflow:step: 3341, g_loss: 2.290, d_loss: 0.289
INFO:tensorflow:step: 3342, g_loss: 3.473, d_loss: 0.468
INFO:tensorflow:step: 3343, g_loss: 1.798, d_loss: 0.324
INFO:tensorflow:step: 3344, g_loss: 2.612, d_loss: 0.283
INFO:tensorflow:step: 3345, g_loss: 1.355, d_loss: 0.546
INFO:tensorflow:step: 3346, g_loss: 6.172, d_loss: 1.027
INFO:tensorflow:step: 3347, g_loss: 0.540, d_loss: 1.213
INFO:tensorflow:step: 3348, g_loss: 6.387, d_loss: 1.981
INFO:tensorflow:step: 3349, g_loss: 0.461, d_loss: 1.347
INFO:tensorflow:step: 3350, g_loss: 5.626, d_loss: 1.684
INFO:tensorflow:step: 3351, g_l

INFO:tensorflow:step: 3478, g_loss: 0.376, d_loss: 1.785
INFO:tensorflow:step: 3479, g_loss: 3.704, d_loss: 1.712
INFO:tensorflow:step: 3480, g_loss: 0.877, d_loss: 1.088
INFO:tensorflow:step: 3481, g_loss: 2.380, d_loss: 0.521
INFO:tensorflow:step: 3482, g_loss: 3.178, d_loss: 0.872
INFO:tensorflow:step: 3483, g_loss: 1.306, d_loss: 0.570
INFO:tensorflow:step: 3484, g_loss: 2.105, d_loss: 0.305
INFO:tensorflow:step: 3485, g_loss: 3.501, d_loss: 0.758
INFO:tensorflow:step: 3486, g_loss: 0.646, d_loss: 1.113
INFO:tensorflow:step: 3487, g_loss: 2.716, d_loss: 0.382
INFO:tensorflow:step: 3488, g_loss: 3.879, d_loss: 0.661
INFO:tensorflow:step: 3489, g_loss: 0.835, d_loss: 0.980
INFO:tensorflow:step: 3490, g_loss: 3.137, d_loss: 0.688
INFO:tensorflow:step: 3491, g_loss: 2.240, d_loss: 0.378
INFO:tensorflow:step: 3492, g_loss: 1.422, d_loss: 0.539
INFO:tensorflow:step: 3493, g_loss: 4.646, d_loss: 1.541
INFO:tensorflow:step: 3494, g_loss: 0.151, d_loss: 2.624
INFO:tensorflow:step: 3495, g_l